<a href="https://colab.research.google.com/github/Arnisha-Akter/Multi-Staged-Approach-for-Automated-Video-Segmentation/blob/main/English_clustering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers

# ELMo

In [ ]:
# Install required packages
!pip install h5py

# Clone the ELMoForManyLangs repository
!git clone https://github.com/HIT-SCIR/ELMoForManyLangs.git

# Navigate into the repository
%cd ELMoForManyLangs

# Install the package
!python setup.py install


Cloning into 'ELMoForManyLangs'...
remote: Enumerating objects: 212, done.
remote: Counting objects: 100% (38/38), done.
remote: Compressing objects: 100% (32/32), done.
remote: Total 212 (delta 9), reused 22 (delta 5), pack-reused 174
Receiving objects: 100% (212/212), 125.80 KiB | 3.31 MiB/s, done.
Resolving deltas: 100% (96/96), done.
/content/ELMoForManyLangs
running install
/usr/local/lib/python3.10/dist-packages/setuptools/_distutils/cmd.py:66: SetuptoolsDeprecationWarning: setup.py install is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` directly.
        Instead, use pypa/build, pypa/installer, pypa/build or
        other standards-based tools.

        See https://blog.ganssle.io/articles/2021/10/setup-py-deprecated.html for details.
        ********************************************************************************

!!
  self.initialize_options()
/usr/local/lib/python3.10

In [ ]:
# Download the English model
!wget http://vectors.nlpl.eu/repository/11/144.zip -O english_model.zip
!unzip english_model.zip -d english_model

# Ensure the model directory is correct
!ls english_model


--2024-05-27 17:20:20--  http://vectors.nlpl.eu/repository/11/144.zip
Resolving vectors.nlpl.eu (vectors.nlpl.eu)... 129.240.189.181
Connecting to vectors.nlpl.eu (vectors.nlpl.eu)|129.240.189.181|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 412990163 (394M) [application/zip]
Saving to: ‘english_model.zip’

english_model.zip   100%[===================>] 393.86M  24.8MB/s    in 17s     

2024-05-27 17:20:37 (23.4 MB/s) - ‘english_model.zip’ saved [412990163/412990163]

Archive:  english_model.zip
  inflating: english_model/char.dic  
  inflating: english_model/config.json  
  inflating: english_model/encoder.pkl  
  inflating: english_model/meta.json  
  inflating: english_model/README    
  inflating: english_model/token_embedder.pkl  
  inflating: english_model/word.dic  
char.dic  config.json  encoder.pkl  meta.json  README  token_embedder.pkl  word.dic


In [ ]:
# The model directory should be something like `144`
import os
model_directory = '/content/ELMoForManyLangs/english_model'

# Verify the existence of the directory
if not os.path.isdir(model_directory):
    raise FileNotFoundError(f"Model directory not found at {model_directory}")

# Copy the sample configuration file if needed
!cp elmoformanylangs/configs/cnn_50_100_512_4096_sample.json {model_directory}/

# Check the model directory contents
!ls {model_directory}


char.dic			 config.json  meta.json  token_embedder.pkl
cnn_50_100_512_4096_sample.json  encoder.pkl  README	 word.dic


In [ ]:
import json
import os

config_path = os.path.join(model_directory, 'config.json')
sample_config_path = 'cnn_50_100_512_4096_sample.json'

# Check if the config file exists
if not os.path.exists(config_path):
    raise FileNotFoundError(f"Config file not found at {config_path}")

# Load and update the config file
with open(config_path, 'r') as f:
    config = json.load(f)

config['config_path'] = sample_config_path

# Save the updated config file
with open(config_path, 'w') as f:
    json.dump(config, f, indent=2)

# Verify the modification
!cat {config_path}


{
  "seed": 1,
  "gpu": 3,
  "train_path": "/users4/conll18st/raw_text/English/en-20m.raw",
  "valid_path": null,
  "test_path": null,
  "config_path": "cnn_50_100_512_4096_sample.json",
  "word_embedding": null,
  "optimizer": "adam",
  "lr": 0.001,
  "lr_decay": 0.8,
  "model": "/users4/conll18st/elmo/src/final_models/en.model",
  "batch_size": 32,
  "max_epoch": 10,
  "clip_grad": 5,
  "max_sent_len": 20,
  "min_count": 3,
  "max_vocab_size": 150000,
  "save_classify_layer": false,
  "valid_size": 0,
  "eval_steps": 10000
}

In [ ]:
# Define your sentences in an array
sentences = [
    "The West Indies didn't make it out of the first round of the most recent T20 World Cup in Australia in 2022 and failed to even qualify for last year's ICC Men's Cricket World Cup in India.",
    "These returns have the legendary fast bowler confident of a strong showing from the West Indies in the ninth edition of the T20 World Cup, an event they will co-host with the USA.",
    "But Ambrose thinks the West Indies' fortunes are turning under skipper Rovman Powell and pointed to T20I series successes over powerhouses South Africa, India, and England in the last 14 months as evidence of this.",
    "A woman suffered injury in her neck by a bullet fired by an unidentified person in Bogura's Shajahanpur upazila yesterday.",
    "The victim's family members and police said, the incident happened on Bogura-Natore regional highway near Birgram under Shajahanpur upazila around 3:00pm yesterday.",
    "The United States has said the sanctions imposed against Bangladesh's elite force Rab are not being withdrawn.",
    "The issue is with the US Department of Justice which is independent and separate from the White House and the State Department.",
    "Zulekha Khatun, 40, of Gohail village is now undergoing treatment at Dhaka Medical College Hospital, our Bogura correspondent reports quoting her family."
]

# Prepare the input file in conllu format
with open('sample_input.conllu', 'w') as f:
    for i, sentence in enumerate(sentences):
        for j, word in enumerate(sentence.split(), 1):
            f.write(f"{j}\t{word}\t{word}\t_\t_\t_\t_\t_\t_\t_\n")
        f.write("\n")

# Verify the input file contents
!cat sample_input.conllu


1	The	The	_	_	_	_	_	_	_
2	West	West	_	_	_	_	_	_	_
3	Indies	Indies	_	_	_	_	_	_	_
4	didn't	didn't	_	_	_	_	_	_	_
5	make	make	_	_	_	_	_	_	_
6	it	it	_	_	_	_	_	_	_
7	out	out	_	_	_	_	_	_	_
8	of	of	_	_	_	_	_	_	_
9	the	the	_	_	_	_	_	_	_
10	first	first	_	_	_	_	_	_	_
11	round	round	_	_	_	_	_	_	_
12	of	of	_	_	_	_	_	_	_
13	the	the	_	_	_	_	_	_	_
14	most	most	_	_	_	_	_	_	_
15	recent	recent	_	_	_	_	_	_	_
16	T20	T20	_	_	_	_	_	_	_
17	World	World	_	_	_	_	_	_	_
18	Cup	Cup	_	_	_	_	_	_	_
19	in	in	_	_	_	_	_	_	_
20	Australia	Australia	_	_	_	_	_	_	_
21	in	in	_	_	_	_	_	_	_
22	2022	2022	_	_	_	_	_	_	_
23	and	and	_	_	_	_	_	_	_
24	failed	failed	_	_	_	_	_	_	_
25	to	to	_	_	_	_	_	_	_
26	even	even	_	_	_	_	_	_	_
27	qualify	qualify	_	_	_	_	_	_	_
28	for	for	_	_	_	_	_	_	_
29	last	last	_	_	_	_	_	_	_
30	year's	year's	_	_	_	_	_	_	_
31	ICC	ICC	_	_	_	_	_	_	_
32	Men's	Men's	_	_	_	_	_	_	_
33	Cricket	Cricket	_	_	_	_	_	_	_
34	World	World	_	_	_	_	_	_	_
35	Cup	Cup	_	_	_	_	_	_	_
36	in	in	_	_	_	_	_	_	_
37	India.	India.	_	_	_	_	_	_	_

1

In [ ]:
# File: elmoformanylangs/modules/highway.py

import torch
import torch.nn as nn

class Highway(nn.Module):
    def __init__(self, input_dim: int, num_layers: int = 1):
        super(Highway, self).__init__()
        self.num_layers = num_layers
        self.nonlinear = nn.ModuleList([nn.Linear(input_dim, input_dim) for _ in range(num_layers)])
        self.linear = nn.ModuleList([nn.Linear(input_dim, input_dim) for _ in range(num_layers)])
        self.gate = nn.ModuleList([nn.Linear(input_dim, input_dim) for _ in range(num_layers)])

    def forward(self, inputs: torch.Tensor) -> torch.Tensor:  # pylint: disable=arguments-differ
        current_inputs = inputs
        for layer in range(self.num_layers):
            nonlinear_output = torch.relu(self.nonlinear[layer](current_inputs))
            linear_output = self.linear[layer](current_inputs)
            gate = torch.sigmoid(self.gate[layer](current_inputs))
            current_inputs = gate * nonlinear_output + (1 - gate) * linear_output
        return current_inputs


In [ ]:
# Create a directory for output if it doesn't exist
output_directory = './output'
os.makedirs(output_directory, exist_ok=True)

# Specify the output file path
output_file_prefix = os.path.join(output_directory, 'sample_output')

# Run the ELMoForManyLangs model
!python -m elmoformanylangs test \
    --input_format conll \
    --input sample_input.conllu \
    --model {model_directory} \
    --output_prefix {output_file_prefix} \
    --output_format hdf5 \
    --output_layer -1

# Verify the output file
!ls {output_directory}


Traceback (most recent call last):
  File "/usr/lib/python3.10/runpy.py", line 187, in _run_module_as_main
    mod_name, mod_spec, code = _get_module_details(mod_name, _Error)
  File "/usr/lib/python3.10/runpy.py", line 146, in _get_module_details
    return _get_module_details(pkg_main_name, error)
  File "/usr/lib/python3.10/runpy.py", line 110, in _get_module_details
    __import__(pkg_name)
  File "/content/ELMoForManyLangs/elmoformanylangs/__init__.py", line 2, in <module>
    from .elmo import Embedder
  File "/content/ELMoForManyLangs/elmoformanylangs/elmo.py", line 12, in <module>
    from .frontend import create_one_batch
  File "/content/ELMoForManyLangs/elmoformanylangs/frontend.py", line 10, in <module>
    from .modules.token_embedder import ConvTokenEmbedder, LstmTokenEmbedder
  File "/content/ELMoForManyLangs/elmoformanylangs/modules/token_embedder.py", line 8, in <module>
    from .highway import Highway
  File "/content/ELMoForManyLangs/elmoformanylangs/modules/highway

In [ ]:
import h5py
import numpy as np
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

# Load the generated embeddings
embeddings_file = 'sample_output.hdf5'
if not os.path.exists(embeddings_file):
    raise FileNotFoundError(f"Embeddings file not found at {embeddings_file}")

with h5py.File(embeddings_file, 'r') as f:
    embeddings = []
    for sentence in f.values():
        sentence_embedding = np.mean(sentence[:], axis=0)  # Average over the 3 layers
        embeddings.append(sentence_embedding)

# Convert embeddings to numpy array
embeddings = np.array(embeddings)

# Normalize the embeddings
scaler = StandardScaler()
normalized_embeddings = scaler.fit_transform(embeddings)

# Perform clustering
num_clusters = 3  # Define the number of clusters (adjust based on your data)
kmeans = KMeans(n_clusters=num_clusters, random_state=0).fit(normalized_embeddings)

# Print clustering results
for i, sentence in enumerate(sentences):
    print(f"Sentence: {sentence} => Cluster: {kmeans.labels_[i]}")


FileNotFoundError: Embeddings file not found at sample_output.hdf5

# **My New Code  (XLNet)**

In [ ]:
!pip install transformers

In [ ]:
import pandas as pd
import random

# Define topics and corresponding sentences
topics = ['AI', 'Technology', 'Weather', 'Health', 'Sports', 'Entertainment', 'IT', 'Movies', 'Politics', 'Data Science']
ai_sentences = [
    "Artificial intelligence (AI) is a wide-ranging tool that enables people to rethink how we integrate information, analyze data, and use the resulting insights to improve decision-making and already it is transforming every walk of life.",
    "In this report, Darrell West and John Allen discuss AI’s application across a variety of sectors, address issues in its development, and offer recommendations for getting the most out of AI while still protecting important human values.",
    "Artificial intelligence algorithms are designed to make decisions, often using real-time data.",
    "They are unlike passive machines that are capable only of mechanical or predetermined responses.",
    "Using sensors, digital data, or remote inputs, they combine information from a variety of different sources, analyze the material instantly, and act on the insights derived from those data.",
    "With massive improvements in storage systems, processing speeds, and analytic techniques, they are capable of tremendous sophistication in analysis and decision-making.",
    "AI generally is undertaken in conjunction with machine learning and data analytics.",
    "Machine learning takes data and looks for underlying trends.",
    "If it spots something that is relevant for a practical problem, software designers can take that knowledge and use it to analyze specific issues.",
    "All that is required are data that are sufficiently robust that algorithms can discern useful patterns.",
    "Data can come in the form of digital information, satellite imagery, visual information, text, or unstructured data."
]

technology_sentences = [
    "Technology is what makes gadgets work.",
    "Gadgets are small things that improve our lives in small ways - a game or a phone.",
    "But technology can be used in ways that improve our lives in big ways - a TV station or medical equipment.",
    "The difference depends on the use we make of technology.",
    "Some examples of technological gadgets include smartphones, tablets, smartwatches, fitness trackers, virtual reality headsets, drones, Bluetooth speakers, digital cameras, e-readers, and gaming consoles.",
    "These gadgets have transformed the way we communicate, work, exercise, and entertain ourselves.",
    "What is a tech gadget?",
    "An umbrella term for any computer-driven device, which today means thousands of products from tablets to toasters.",
    "Information Access: Gadgets provide easy access to a vast amount of information on the internet.",
    "This helps in learning, research, and staying informed about various topics.",
    "Productivity and Efficiency: Electronic gadgets like laptops and tablets enable people to work efficiently, manage tasks, and organize schedules."
]

weather_sentences = [
    "There are six main components, or parts, of weather.",
    "They are temperature, atmospheric pressure, wind, humidity, precipitation, and cloudiness.",
    "Together, these components describe the weather at any given time.",
    "These changing components, along with the knowledge of atmospheric processes, help meteorologists—scientists who study weather—forecast what the weather will be in the near future.",
    "Temperature is measured with a thermometer and refers to how hot or cold the atmosphere is.",
    "Meteorologists report temperature two ways: in Celsius (C) and Fahrenheit (F).",
    "The United States uses the Fahrenheit system; in other parts of the world, Celsius is used.",
    "Almost all scientists measure temperature using the Celsius scale.",
    "Temperature is a relative measurement.",
    "An afternoon at 70 degrees Fahrenheit, for example, would seem cool after several days of 95 degrees Fahrenheit, but it would seem warm after temperatures around 32 degrees Fahrenheit.",
    "The coldest weather usually happens near the poles, while the warmest weather usually happens near the Equator.",
    "Atmospheric pressure is the weight of the atmosphere overhead.",
    "Changes in atmospheric pressure signal shifts in the weather.",
    "A high-pressure system usually brings cool temperatures and clear skies.",
    "A low-pressure system can bring warmer weather, storms, and rain."
]

health_sentences = [
    "Maintaining a balanced diet is essential for overall health and well-being.",
    "Regular exercise can help improve cardiovascular health and reduce the risk of chronic diseases.",
    "Getting enough sleep is crucial for physical and mental health.",
    "Staying hydrated is important for maintaining energy levels and proper body function.",
    "Mental health is just as important as physical health, and it's important to take steps to manage stress and anxiety.",
    "Routine health check-ups can help detect and prevent potential health issues early on.",
    "Practicing good hygiene, such as regular handwashing, can prevent the spread of illnesses.",
    "Engaging in relaxation techniques like meditation or yoga can improve mental health and reduce stress.",
    "A healthy lifestyle includes both physical activity and proper nutrition.",
    "Avoiding smoking and limiting alcohol consumption can significantly improve long-term health.",
    "Social connections and a strong support system can positively impact mental and emotional health.",
    "Regularly participating in activities that you enjoy can enhance your overall quality of life.",
    "Maintaining a healthy weight is important for reducing the risk of various health conditions.",
    "Sun protection, such as wearing sunscreen, can prevent skin damage and reduce the risk of skin cancer.",
    "Being proactive about your health and making informed decisions can lead to a longer, healthier life."
]

sports_sentences = [
    "Bangladesh enter the T20 World Cup with a bit of uncertainty as they continue to look for their strongest XI.",
    "Experienced opener Litton Das has been out of form of late with an average of 13.16 this year in six matches.",
    "Although teammate Tawhid Hridoy is confident that Das will make a strong comeback in the tournament.",
    "Another concern is the injury to vice captain Taskin Ahmed and his fitness will be monitored prior to Bangladesh's first contest against Sri Lanka on 7 June after he suffered a side strain before the final match of the recent series against Zimbabwe.",
    "The Tigers have backed experience.",
    "Other than Das, they have former Bangladesh skippers Mahmudullah and Shakib Al Hasan who can provide Najmul Hossain Shanto much-needed support as he undertakes his first ICC tournament as the new captain.",
    "Bangladesh vice-captain Taskin Ahmed said the out-of-form of two senior players clearly had an effect on the team but not off the field",
    "India beat South Africa by seven runs in the final at Kensington Oval on Saturday to win the Twenty20 World Cup title."
]

entertainment_sentences = [
    "Taylor Swift will be playing three nights at Edinburgh’s Murrayfield stadium from 7 to 9 June.",
    "Hundreds of thousands of fans are set to flock to the city, and competition for hotel rooms is fierce.",
    "The housing charity said people being declared homeless in Edinburgh who would usually be offered temporary accommodation such as hotels were being sent out of the city due to a severe shortage in accommodation caused by the concert.",
    "There is no evidence of homeless people being removed from accommodation where they are already staying.",
    "For the first time, Taylor Swift performed The Albatross live at the Eras Tour in a packed Dublin, Ireland stadium as a surprise song for fans.",
    "The singer-songwriter has 44 more shows left on her mammoth Eras Tour including one more night in the Irish capital."
]

it_sentences = [
    "In the last few decades, IT professionals have become the keystone of modern society, frequently creating and supporting products and services that have become a critical part of our everyday lives.",
    "National IT Professionals Day is celebrated annually on the third Tuesday of every September to remember the skilled and hardworking people who keep systems and networks running smoothly worldwide.",
    "The world’s dependence on computers and software has only expanded exponentially in recent decades, highlighting the importance of these professionals.",
    "Here are our top ten article recommendations on IT careers you should read this year to encourage your IT teams to do what they do best.",
    "IT professionals are the backbone of any modern organization, ensuring that critical infrastructure is secure, efficient, and up-to-date.",
    "Their expertise spans a wide range of areas, including cybersecurity, network management, software development, and technical support.",
    "With the rapid pace of technological change, continuous learning and skill development are essential for IT professionals to stay ahead in their field.",
    "The demand for IT expertise is growing, driven by the increasing complexity of technology and the need for businesses to stay competitive.",
    "Many organizations offer professional development programs and certifications to help IT professionals advance their careers.",
    "IT professionals often work behind the scenes, but their contributions are vital to the success of almost every sector, from healthcare to finance.",
    "In addition to technical skills, IT professionals need strong problem-solving abilities and excellent communication skills to effectively manage and support diverse technologies.",
    "The role of IT professionals is not just about maintaining systems, but also about driving innovation and implementing new solutions to improve business operations.",
    "As remote work becomes more prevalent, IT professionals are crucial in ensuring secure and reliable access to company resources for employees worldwide.",
    "Recognizing the hard work and dedication of IT professionals can boost morale and encourage them to continue their vital work with enthusiasm."
]

movies_sentences = [
    "Movies play a significant role in shaping popular culture and societal norms.",
    "They entertain, inform, and inspire audiences around the world.",
    "From Hollywood blockbusters to independent films, the movie industry is a diverse and dynamic art form.",
    "In recent years, the rise of streaming services has transformed how we consume movies, allowing viewers to access a vast library of films from the comfort of their own homes.",
    "Despite these changes, the cinema remains a powerful medium for storytelling and artistic expression.",
    "Whether it's a big-budget action flick or a thought-provoking indie drama, movies have the ability to captivate audiences and spark meaningful conversations.",
    "What is your favorite movie genre, and why?",
    "Do you prefer watching movies in theaters or at home?",
    "Who is your favorite actor or actress, and what do you admire most about their work?"
]

politics_sentences = [
    "Politics plays a crucial role in shaping the direction of society and the lives of its citizens.",
    "From local elections to international diplomacy, political decisions have far-reaching implications.",
    "However, the world of politics can be complex and contentious, with competing interests and ideologies often clashing.",
    "In recent years, issues such as climate change, income inequality, and social justice have become central topics of political debate.",
    "As citizens, it's important to stay informed about political issues and participate in the democratic process.",
    "What political issues are most important to you, and why?",
    "Do you think politicians are doing enough to address the challenges facing society today?",
    "How can we encourage more people to engage with politics and participate in elections?"
]

data_science_sentences = [
    "Data science is not a new term in the world of technology today.",
    "It has taken over the corporate world in the last few years.",
    "The demand for skilled data science professionals has seen an upsurge, as organizations are on the constant lookout for data science professionals to resolve business complexities with efficient data analysis.",
    "Because data science is broad, with methods drawing from computer science, statistics, and other disciplines and with applications appearing in all sectors, these challenge areas speak to the breadth of issues spanning science, technology",
    "Data science has become integral to decision-making processes across various industries, from finance to healthcare, and even entertainment.",
    "Companies are investing heavily in data infrastructure and analytics tools to harness the power of big data.",
    "The field encompasses a wide range of techniques and technologies, including machine learning, statistical analysis, and data visualization.",
    "Data scientists play a crucial role in transforming raw data into actionable insights that drive strategic initiatives.",
    "With advancements in artificial intelligence and machine learning, data science is continuously evolving, opening new possibilities for innovation.",
    "Educational institutions are increasingly offering specialized programs and degrees in data science to meet the growing demand for expertise.",
    "The role of a data scientist is multidisciplinary, requiring knowledge of programming, mathematics, and domain-specific expertise.",
    "Open-source tools like Python and R have made data science more accessible to a broader audience, encouraging more individuals to enter the field.",
    "Data science projects often involve collaboration between various stakeholders, including business analysts, IT professionals, and domain experts.",
    "The ethical implications of data usage and privacy are becoming more prominent as data science continues to expand its reach."
]

# Generate synthetic dataset
data = []
for _ in range(1000):  # Generate 1000 samples
    topic = random.choice(topics)
    if topic == 'AI':
        sentence = random.choice(ai_sentences)
    elif topic == 'Technology':
        sentence = random.choice(technology_sentences)
    elif topic == 'Weather':
        sentence = random.choice(weather_sentences)
    elif topic == 'Health':
        sentence = random.choice(health_sentences)
    elif topic == 'Sports':
        sentence = random.choice(sports_sentences)
    elif topic == 'Entertainment':
        sentence = random.choice(entertainment_sentences)
    elif topic == 'IT':
        sentence = random.choice(it_sentences)
    elif topic == 'Movies':
        sentence = random.choice(movies_sentences)
    elif topic == 'Politics':
        sentence = random.choice(politics_sentences)
    elif topic == 'Data Science':
        sentence = random.choice(data_science_sentences)
    else:
        sentence = "others"
    data.append((sentence, topic))

# Shuffle the data
random.shuffle(data)

# Create a DataFrame
df = pd.DataFrame(data, columns=['sentence', 'topic'])

# Print the count of sentences and array length
print("Number of sentences:", len(data))
print("DataFrame shape:", df.shape)

# Save the dataset to a CSV file
df.to_csv('topic_classification_dataset.csv', index=False)


Number of sentences: 1000
DataFrame shape: (1000, 2)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import XLNetTokenizer, XLNetForSequenceClassification, AdamW
from torch.utils.data import DataLoader, Dataset
import torch
from tqdm import tqdm

# Step 1: Data Preprocessing
# Load the dataset
df = pd.read_csv('topic_classification_dataset.csv')

# Drop rows with NaN values in the 'sentence' column
df = df.dropna(subset=['sentence'])

# Split the dataset into training, validation, and test sets
train_texts, val_texts, train_labels, val_labels = train_test_split(df['sentence'], df['topic'], test_size=0.2, random_state=42)
val_texts, test_texts, val_labels, test_labels = train_test_split(val_texts, val_labels, test_size=0.5, random_state=42)

# Ensure text data is in list format
train_texts = train_texts.tolist()
val_texts = val_texts.tolist()
test_texts = test_texts.tolist()

# Check if all elements in train_texts, val_texts, and test_texts are strings
def check_texts(texts, name):
    for i, text in enumerate(texts):
        if not isinstance(text, str):
            print(f"Non-string element found in {name} at index {i}: {text} (type: {type(text)})")

check_texts(train_texts, "train_texts")
check_texts(val_texts, "val_texts")
check_texts(test_texts, "test_texts")

# Load XLNet tokenizer
tokenizer = XLNetTokenizer.from_pretrained('xlnet-base-cased')

# Tokenize the sentences with a specified max_length
max_length = 128  # Define the maximum length for tokenization
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=max_length, return_tensors='pt')
val_encodings = tokenizer(val_texts, truncation=True, padding=True, max_length=max_length, return_tensors='pt')
test_encodings = tokenizer(test_texts, truncation=True, padding=True, max_length=max_length, return_tensors='pt')

# Convert labels to integers
label_map = {label: i for i, label in enumerate(df['topic'].unique())}
train_labels = [label_map[label] for label in train_labels]
val_labels = [label_map[label] for label in val_labels]
test_labels = [label_map[label] for label in test_labels]

# Define PyTorch dataset
class TopicDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx], dtype=torch.long)
        return item

    def __len__(self):
        return len(self.labels)

# Create dataset objects
train_dataset = TopicDataset(train_encodings, train_labels)
val_dataset = TopicDataset(val_encodings, val_labels)
test_dataset = TopicDataset(test_encodings, test_labels)

# Step 2: Fine-tuning the Model
# Load XLNet model
model = XLNetForSequenceClassification.from_pretrained('xlnet-base-cased', num_labels=len(label_map))

# Define training parameters
optimizer = AdamW(model.parameters(), lr=5e-5)

# Define training loop
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)

num_epochs = 3
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for batch in tqdm(train_loader, desc=f"Training Epoch {epoch + 1}/{num_epochs}"):
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        total_loss += loss.item()
        loss.backward()
        optimizer.step()
    print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {total_loss / len(train_loader)}")

# Step 3: Evaluation
# Evaluation loop
model.eval()
correct = 0
total = 0
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False)
with torch.no_grad():
    for batch in tqdm(val_loader, desc="Validation"):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        outputs = model(input_ids, attention_mask=attention_mask)
        predictions = torch.argmax(outputs.logits, dim=1)
        total += labels.size(0)
        correct += (predictions == labels).sum().item()

print(f"Validation Accuracy: {correct / total}")

# Step 4: Inference
# Example of using the fine-tuned model for inference
example_sentence = "The new iPhone is rumored to have groundbreaking features."
inputs = tokenizer(example_sentence, truncation=True, padding=True, max_length=max_length, return_tensors="pt")
inputs = {key: val.to(device) for key, val in inputs.items()}
outputs = model(**inputs)
predicted_label = torch.argmax(outputs.logits, dim=1).item()
predicted_topic = [topic for topic, label in label_map.items() if label == predicted_label][0]
print(f"Predicted Topic for '{example_sentence}': {predicted_topic}")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


spiece.model:   0%|          | 0.00/798k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.38M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/760 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/467M [00:00<?, ?B/s]

...

Epoch 2/3, Loss: 2.3275326228141786


Training Epoch 3/3: 100%|██████████| 100/100 [00:10<00:00,  9.66it/s]


Epoch 3/3, Loss: 2.344857223033905


Validation: 100%|██████████| 13/13 [00:00<00:00, 31.99it/s]


Validation Accuracy: 0.12
Predicted Topic for 'The new iPhone is rumored to have groundbreaking features.': Health


In [ ]:
# Step 4: Inference
# Example of using the fine-tuned model for inference
example_sentence = "Heavy rain is expected in Dhaka tomorrow"

# Tokenize the input sentence with a specified max_length
inputs = tokenizer(example_sentence, truncation=True, padding='max_length', max_length=max_length, return_tensors="pt")

# Move the input tensors to the same device as the model
inputs = {key: val.to(device) for key, val in inputs.items()}

# Set the model to evaluation mode and perform inference
model.eval()
with torch.no_grad():
    outputs = model(**inputs)
    predicted_label = torch.argmax(outputs.logits, dim=1).item()

# Map the predicted label back to the corresponding topic
predicted_topic = [topic for topic, label in label_map.items() if label == predicted_label][0]
print(f"Predicted Topic for '{example_sentence}': {predicted_topic}")


Predicted Topic for 'Heavy rain is expected in Dhaka tomorrow': Weather


In [ ]:
# Step 4: Inference with multiple examples
# Example sentences covering various topics
example_sentences = [
    "The new iPhone is rumored to have groundbreaking features.",  # Technology
    "Heavy rain is expected tomorrow.",                            # Weather
    "The football team won the championship.",                     # Sports
    "The President of Bangladesh will address the nation.",        # Politics
    "The movie received rave reviews from critics.",               # Entertainment
    "Bangladesh's first Data Science Program opens up in UIU.",    # Education
    "The stock market hit a record high today.",                   # Finance
    "Researchers discovered a new species of dinosaur.",           # Science
    "The local bakery introduced a new line of gluten-free cakes.",# Food
    "A new law was passed to protect wildlife.",                   # Environment
    "An art exhibit featuring modern artists opened downtown.",    # Art
    "The new software update improves performance significantly.", # Technology
    "A severe thunderstorm warning has been issued.",              # Weather
    "The national basketball team is preparing for the Olympics.", # Sports
    "The Prime Minister announced new economic reforms.",          # Politics
    "The latest episode of the TV show broke viewership records.", # Entertainment
    "The university is offering a new course in data science.",    # Education
    "The company reported a substantial increase in quarterly earnings.", # Finance
    "Astronomers discovered a potentially habitable exoplanet.",   # Science
    "The new restaurant in town is getting rave reviews.",         # Food
    "The government launched a campaign to reduce plastic waste.", # Environment
    "A famous painter's work is being displayed at the museum.",    # Art
    "Because data science is broad, with methods drawing from computer science, statistics, and other disciplines and with applications appearing in all sectors, these challenge areas speak to the breadth of issues spanning science, technology, and society." #DS
]

# Tokenize and classify each sentence
for sentence in example_sentences:
    inputs = tokenizer(sentence, return_tensors="pt", truncation=True, padding='max_length', max_length=max_length)
    inputs = {key: val.to(device) for key, val in inputs.items()}
    model.eval()
    with torch.no_grad():
        outputs = model(**inputs)
        predicted_label = torch.argmax(outputs.logits, dim=1).item()
    predicted_topic = [topic for topic, label in label_map.items() if label == predicted_label][0]
    print(f"Predicted Topic for '{sentence}': {predicted_topic}")


Predicted Topic for 'The new iPhone is rumored to have groundbreaking features.': Health
Predicted Topic for 'Heavy rain is expected tomorrow.': Health
Predicted Topic for 'The football team won the championship.': Health
Predicted Topic for 'The President of Bangladesh will address the nation.': Health
Predicted Topic for 'The movie received rave reviews from critics.': Health
Predicted Topic for 'Bangladesh's first Data Science Program opens up in UIU.': Health
Predicted Topic for 'The stock market hit a record high today.': Health
Predicted Topic for 'Researchers discovered a new species of dinosaur.': Health
Predicted Topic for 'The local bakery introduced a new line of gluten-free cakes.': Health
Predicted Topic for 'A new law was passed to protect wildlife.': Health
Predicted Topic for 'An art exhibit featuring modern artists opened downtown.': Health
Predicted Topic for 'The new software update improves performance significantly.': Health
Predicted Topic for 'A severe thundersto

In [ ]:
torch.save(model.state_dict(), '/content/drive/MyDrive/XLNet.pth')